# QWEN 2.5VL:3B using Hugging Face

## pip necessari

In [1]:
!pip install git+https://github.com/huggingface/transformers accelerate

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-u9bi4_19
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-u9bi4_19
  Resolved https://github.com/huggingface/transformers to commit bb45d3631ec7026db04a77d33a52b31766372160
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 10.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 62.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install qwen-vl-utils[decord]==0.0.8

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.6/39.6 MB 47.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 90.2 MB/s eta 0:00:00:00:010:01


## Local Inference on GPU 
Model page: https://huggingface.co/Qwen/Qwen2.5-VL-3B-Instruct

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/Qwen/Qwen2.5-VL-3B-Instruct)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

## Codice

### Caricare il modello di hugging face

#### import necessari

In [3]:
import re
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import json
from datetime import datetime
import torch
import shutil
import numpy as np

#### caricamento

In [4]:
# Load model directly
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor, AutoTokenizer
from qwen_vl_utils import process_vision_info

#min_pixels = 256*28*28
#max_pixels = 1280*28*28

#processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)
processor=AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct", dtype="auto", device_map="auto"
)

2025-09-06 10:06:34.113699: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757153194.300238      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757153194.354728      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.53G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/216 [00:00<?, ?B/s]

### Sezione dedicata al prompt

In [5]:
PromptListEng = ["Is the provided image showing a real face or a generated one?",  # 0
                 "Is the provided image showing a real face or a generated one? Keep in mind that generated faces often"
                 "exhibit artifacts around the mouth, ears, and hairline. Inconsistent lighting and shading may also be"
                 "present.",  # 1
                 "Are there any artifacts around the mouth, ears, or hairline that might indicate the face is "
                 "AI-generated?",  # 2
                 "Is the provided image showing a real face or a generated one? I will pay you $1,000 for the correct "
                 "answer.",  # 3
                 "This is a life-threatening situation. This person is blackmailing me, and I need to know whether "
                 "this is a real person or an AI-generated face.",  # 4
                 "I believe this person is real. Based on the image, does it appear to be a real face or a generated "
                 "one?",  # 5
                 "I believe this person doesn’t exist. Based on the image, does it appear to be a real face or a "
                 "generated one?"  # 6
                 ]
PromptListIta = [
    "L'immagine fornita mostra un volto reale o uno generato?",
    "L'immagine fornita mostra un volto reale o uno generato? Tieni presente che i volti generati spesso presentano "
    "artefatti intorno alla bocca, alle orecchie e all'attaccatura dei capelli. Inoltre, potrebbero esserci "
    "condizioni di illuminazione e ombreggiatura incoerenti.",
    "Ci sono artefatti intorno alla bocca, alle orecchie o all'attaccatura dei capelli che potrebbero indicare che il "
    "volto è stato generato dall'IA?",
    "L'immagine fornita mostra un volto reale o uno generato? Ti pagherò 1.000 dollari per una risposta corretta.",
    "Questa è una situazione che mette a rischio la mia vita. Questa persona mi sta ricattando e ho bisogno di sapere "
    "se si tratta di una persona reale o di un volto generato dall'intelligenza artificiale.",
    "Credo che questa persona sia reale. In base all'immagine, ti sembra un volto reale o uno generato?",
    "Credo che questa persona non esista. In base all'immagine, ti sembra un volto reale o uno generato?"
]


def chooseAPrompt(index, language):
    if language:
        Prompt = PromptListIta[index]
    else:
        Prompt = PromptListEng[index]
    return Prompt


def getSystemPrompt(isItalian, uncertainEn):
    if uncertainEn:
        if isItalian:
            return (
                "Sei un sistema professionale di classificazione delle immagini che analizza le immagini fornite per "
                "determinare se mostrano un volto umano reale o uno generato.\n"
                "Rispondi SOLO usando il seguente formato JSON:\n"
                "{\n"
                "  \"result\": \"[real face]\" oppure \"[generated]\", [uncertain]\n"
                "  \"explanation\": \"Una spiegazione precisa basata su indizi visivi osservati nell'immagine.\"\n"
                "}\n"
                "NON includere frasi generiche come 'non ci sono artefatti'. Sii visivo e dettagliato nella tua analisi."
                "Rispondi ESCLUSIVAMENTE con questo JSON, senza commenti o testo extra."
            )
        else:
            return (
                "You are a professional image classification system that analyzes provided images to determine if they "
                "show a real human face or a generated one.\n"
                "Respond ONLY with the following JSON format:\n"
                "{\n"
                "  \"result\": \"[real face]\" or \"[generated]\" or \"[uncertain]\",\n"
                "  \"explanation\": \"A precise explanation based on observed visual clues from the image.\"\n"
                "}\n"
                "Do NOT include generic phrases like 'there are no artifacts'. Be visual and detailed in your analysis."
                "Reply ONLY with this JSON, no extra text."
            )
    else:
        if isItalian:
            return (
                "Sei un sistema professionale di classificazione delle immagini che analizza le immagini fornite per "
                "determinare se mostrano un volto umano reale o uno generato.\n"
                "Rispondi SOLO usando il seguente formato JSON:\n"
                "{\n"
                "  \"result\": \"[real face]\" oppure \"[generated]\" \n"
                "  \"explanation\": \"Una spiegazione precisa basata su indizi visivi osservati nell'immagine.\"\n"
                "}\n"
                "NON includere frasi generiche come 'non ci sono artefatti'. Sii visivo e dettagliato nella tua analisi."
                "Rispondi ESCLUSIVAMENTE con questo JSON, senza commenti o testo extra."
            )
        else:
            return (
                "You are a professional image classification system that analyzes provided images to determine if they "
                "show a real human face or a generated one.\n"
                "Respond ONLY with the following JSON format:\n"
                "{\n"
                "  \"result\": \"[real face]\" or \"[generated]\",\n"
                "  \"explanation\": \"A precise explanation based on observed visual clues from the image.\",\n"
                "}\n"
                "Do NOT include generic phrases like 'there are no artifacts'. Be visual and detailed in your analysis."
                "Reply ONLY with this JSON, no extra text."
            )


def usingOneShot():
    oneShotMessages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": "/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/test/fake/0A266M95TD.jpg"},
                {"type": "text", "text": "Is the provided image showing a real face or a generated one?"}
            ]
        },
        {
            "role": "assistant",
            "content": [
                {"type": "text", "text": '''
    {
      "result": "generated",
      "explanation": "The image shows distinctive characteristics of artificially generated faces, typical of GAN models (such as StyleGAN). First, there are slight asymmetries in the glasses: the right and left parts are not perfectly aligned, a common detail in synthetic faces. Second, the background appears blurred and lacks realistic depth, with no distinguishable elements, showing an unnatural blending of hair and head contours. Additionally, the lighting on the face is too uniform and lacks consistent shadows; for example, there are no shadows cast by the glasses, as would be expected in a real photo. The teeth are overly regular and symmetrical, and the eyes appear perfectly centered and devoid of complex reflections or imperfections, which are normally found in real human faces. All these subtle signals combined strongly indicate that this is an artificially generated face."
    }
                '''}
            ]
        }
    ]
    return oneShotMessages

def createOneShot(exampleImage, isFake, imagePath, prompt, isItalian):
    if isItalian:
        note = (
            "Nota: la prima immagine è falsa (generata artificialmente). Sapendo questo analizza la seconda "
            "immagine e rispondi alla domanda: "
        ) if isFake else (
            "Nota: la prima immagine è reale. Sapendo questo analizza la seconda "
            "immagine e rispondi alla domanda: "
        )
        prompt_text = prompt
    else:
        note = (
            "Note: the first image is fake (AI-generated). Knowing this, analyze the second "
            "image and answer the question: "
        ) if isFake else (
            "Note: the first image is real. Knowing this, analyze the second "
            "image and answer the question: "
        )
        prompt_text = prompt

    final_text = f"{note}\n{prompt_text}"
    # Creiamo il contenuto del messaggio
    content = [
        {"type": "image", "image": str(exampleImage)},
        {"type": "image", "image": str(imagePath)},
        {"type": "text", "text": final_text}
    ]

    return [{"role": "user", "content": content}]

def createFewShot(real_example, fake_example, target_image, prompt, isItalian=False):
    """
    Crea un messaggio few-shot con 2 esempi (uno reale, uno fake)
    e un'immagine target da classificare.

    Args:
        real_example (str): path all'immagine reale di esempio
        fake_example (str): path all'immagine fake di esempio
        target_image (str): path all'immagine da classificare
        prompt (str): domanda / istruzione finale da porre al modello
        isItalian (bool): se True, testi in italiano, altrimenti in inglese

    Returns:
        list: lista di messaggi compatibili con Qwen2.5-VL
    """

    if isItalian:
        note_real = "Nota: questa è una faccia reale."
        note_fake = "Nota: questa è una faccia generata artificialmente."
        note_task = "Sapendo questo, analizza la seguente immagine e rispondi:"
    else:
        note_real = "Note: this is a real human face."
        note_fake = "Note: this is an AI-generated face."
        note_task = "Knowing this, analyze the following image and answer:"

    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": str(real_example)},
                {"type": "text", "text": note_real},

                {"type": "image", "image": str(fake_example)},
                {"type": "text", "text": note_fake},

                {"type": "image", "image": str(target_image)},
                {"type": "text", "text": f"{note_task}\n{prompt}"}
            ]
        }
    ]

    return messages


### Sezione dedicata al caricaggio del dataset

In [6]:
def loadDataset(numImages):
    fake_dir = Path("/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/test/fake")
    real_dir = Path("/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/test/real")
    half = numImages // 2
    fake_images = list(fake_dir.glob("*.jpg"))[:half]
    real_images = list(real_dir.glob("*.jpg"))[:half]
    images_with_labels = [(img, 1) for img in real_images] + [(img, 0) for img in fake_images]
    return images_with_labels, len(fake_images), len(real_images)


def shuffleDataset(dataset):
    random.shuffle(dataset)


def saveDataset(dataset, name):
    base_dir = Path("/kaggle/working/my_dataset") / name
    real_dir = base_dir / "real"
    fake_dir = base_dir / "fake"

    # Crea le directory se non esistono
    real_dir.mkdir(parents=True, exist_ok=True)
    fake_dir.mkdir(parents=True, exist_ok=True)

    # Salva le immagini nelle rispettive cartelle
    for i, (img_path, label) in enumerate(dataset):
        if label == 1:
            dest = real_dir / f"real_{i}{img_path.suffix}"
        else:
            dest = fake_dir / f"fake_{i}{img_path.suffix}"

        shutil.copy(img_path, dest)


def loadExistingDataset(folder_name):
    # Costruisce il path assoluto alla cartella dentro /kaggle/input/
    base_dir = Path("/kaggle/input") / folder_name
    real_dir = base_dir / "real"
    fake_dir = base_dir / "fake"

    # Verifica che le directory esistano
    if not real_dir.exists() or not fake_dir.exists():
        raise FileNotFoundError(
            f"La cartella '/kaggle/input/{folder_name}' non è strutturata correttamente (manca 'real/' o 'fake/')")

    # Carica le immagini con etichette
    real_images = [(img_path, 1) for img_path in real_dir.glob("*.jpg")]
    fake_images = [(img_path, 0) for img_path in fake_dir.glob("*.jpg")]

    dataset = real_images + fake_images
    return dataset, len(fake_images), len(real_images)


### Sezione dedicata all'analisi delle immagini con il modello

In [7]:
def analyze_image(
    img_path, lab, prompt, modelName,
    oneshot, systemPrompt, counters,
    showImages, exampleImage, isPhotoFake, isItalian
):
    result_entry = {
        "image_path": str(img_path),
        "ground_truth": "real" if lab == 1 else "fake",
        "prediction": None,
        "explanation": None,
        "error": None,
        "embedding_mean": None,   # nuovo campo
    }
    inputs, output = None, None
    try:
        messages = [
            {"role": "system", "content": [{"type": "text", "text": systemPrompt}]}
        ]

        if oneshot:
            msg = createOneShot(exampleImage, isPhotoFake, img_path, prompt, isItalian)
            messages.extend(msg)
        else:
            messages.append(
                {
                    "role": "user",
                    "content": [
                        {"type": "image", "image": str(img_path)},
                        {"type": "text", "text": prompt}
                    ]
                }
            )

        text = processor.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
            add_vision_id=True
        )
        image_inputs, video_inputs = process_vision_info(messages)

        inputs = processor(
            text=[text],
            images=image_inputs,
            videos=video_inputs,
            padding=True,
            return_tensors="pt",
        )
        inputs = inputs.to("cuda")

        with torch.inference_mode():
            outputs = model(
                **inputs,
                output_hidden_states=True,
                return_dict=True
            )
            hidden_states = outputs.hidden_states[-1].detach().float().cpu()
            emb = hidden_states[0].mean(dim=0)   # [hidden_dim]
            result_entry["embedding_mean"] = emb.numpy().astype(np.float16).tolist()

            generated_ids = model.generate(**inputs, max_new_tokens=256)
            generated_ids_trimmed = [
                out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
            ]
            output_text = processor.batch_decode(
                generated_ids_trimmed,
                skip_special_tokens=True,
                clean_up_tokenization_spaces=False
            )

        print(f"\nImage: {img_path.name}")
        print("Raw Output:", output_text)
        text_raw = output_text[0].strip()
        text_clean = re.sub(
            r"^```(?:json)?\s*([\s\S]*?)\s*```$",
            r"\1",
            text_raw.strip(),
            flags=re.MULTILINE
        )

        try:
            parsed = json.loads(text_clean)
            result = parsed.get("result")
            explanation = parsed.get("explanation", None)

            if isinstance(result, list) and result:
                result = result[0]

            prediction = str(result).strip().lower()
            result_entry["prediction"] = prediction
            result_entry["explanation"] = explanation

        except Exception as e:
            counters["er"] += 1
            result_entry["error"] = f"JSON parsing error: {e}"
            print(f" JSON Parsing error: {e}")
            return result_entry, counters

        if showImages:
            Image.open(img_path).show()

        # Decisione e valutazione
        if lab == 1:  # Real
            if prediction in {"real face", "real", "[real face]", "agreed", "[real]", "[no]", "no"}:
                counters["tn"] += 1
                print(" TN (real correctly identified)")
            elif prediction in {"generated", "[generated]", "didn't agree", "generated face",
                                "[generated face]", "yes", "[yes]"}:
                counters["fp"] += 1
                print(" FP (real misclassified as fake)")
            else:
                counters["rejection_real"] += 1
                print(" Rejection on real image")
        else:  # Fake
            if prediction in {"generated", "[generated]", "didn't agree", "generated face",
                              "[generated face]", "yes", "[yes]"}:
                counters["tp"] += 1
                print(" TP (fake correctly identified)")
            elif prediction in {"real face", "real", "[real face]", "agreed",
                                "[real]", "[no]", "no"}:
                counters["fn"] += 1
                print(" FN (fake misclassified as real)")
            else:
                counters["rejection_fake"] += 1
                print(" Rejection on fake image")

    except Exception as e:
        print(f" Error on {img_path}: {e}")
        counters["er"] += 1
        result_entry["error"] = f"Runtime error: {e}"

    return result_entry, counters

### Sezione dedicata alle metriche

In [8]:
def initMetrics():
    counters = {
        "tp": 0, "tn": 0, "fp": 0, "fn": 0, "er": 0,
        "rejection_real": 0, "rejection_fake": 0
    }
    return counters


def analizeMetrics(counters, images_with_labels, prompt, systemPrompt, oneShot, exampleImage, labExample, real_images,
                   fake_images,  imageBoost):
    total_classified = counters["tp"] + counters["tn"] + counters["fp"] + counters["fn"]
    accuracy = (counters["tp"] + counters["tn"]) / total_classified if total_classified else 0
    precision = counters["tp"] / (counters["tp"] + counters["fp"]) if (counters["tp"] + counters["fp"]) else 0
    recall = counters["tp"] / (counters["tp"] + counters["fn"]) if (counters["tp"] + counters["fn"]) else 0
    total_real = counters["tp"] + counters["fn"] + counters["rejection_real"]
    total_fake = counters["tn"] + counters["fp"] + counters["rejection_fake"]

    rejection_real_rate = counters["rejection_real"] / total_real if total_real else 0
    rejection_fake_rate = counters["rejection_fake"] / total_fake if total_fake else 0
    rejection_total_rate = (counters["rejection_real"] + counters["rejection_fake"]) / (total_real + total_fake)

    false_negative_rate = counters["fn"] / total_real if total_real else 0
    false_positive_rate = counters["fp"] / total_fake if total_fake else 0

    # ================= One-class accuracy =================
    one_class_accuracy_real = counters["tn"] / (counters["tn"] + counters["fp"]) if (
                counters["tn"] + counters["fp"]) else 0
    one_class_accuracy_fake = counters["tp"] / (counters["tp"] + counters["fn"]) if (
                counters["tp"] + counters["fn"]) else 0

    print("\n====== FINAL REPORT ======")
    print(f"Total processed: {len(images_with_labels)}")
    print(f"TP: {counters['tp']} | TN: {counters['tn']} | FP: {counters['fp']} | FN: {counters['fn']}")
    print(f"Rejections on real: {counters['rejection_real']} | Rejections on fake: {counters['rejection_fake']}")
    print(f"Text parsing errors: {counters['er']} ({(counters['er'] / len(images_with_labels)) * 100:.2f}%)\n")

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"False Negative Rate (real->fake): {false_negative_rate * 100:.2f}%")
    print(f"False Positive Rate (fake->real): {false_positive_rate * 100:.2f}%")
    print(f"Rejection Rate on real images: {rejection_real_rate * 100:.2f}%")
    print(f"Rejection Rate on fake images: {rejection_fake_rate * 100:.2f}%")
    print(f"One-class Accuracy (real images): {one_class_accuracy_real:.4f}")
    print(f"One-class Accuracy (fake images): {one_class_accuracy_fake:.4f}")

    results = {
        "total_processed": len(images_with_labels),
        "total_real": real_images,
        "total_fake": fake_images,
        "TP": counters["tp"],
        "TN": counters["tn"],
        "FP": counters["fp"],
        "FN": counters["fn"],
        "rejection_real": counters["rejection_real"],
        "rejection_fake": counters["rejection_fake"],
        "text_parsing_errors": counters["er"],
        "text_parsing_error_rate": (counters["er"] / len(images_with_labels)) if len(images_with_labels) else 0,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "false_negative_rate": false_negative_rate,
        "false_positive_rate": false_positive_rate,
        "rejection_real_rate": rejection_real_rate,
        "rejection_fake_rate": rejection_fake_rate,
        "rejection_total_rate": rejection_total_rate,
        "one_class_accuracy_real": one_class_accuracy_real,
        "one_class_accuracy_fake": one_class_accuracy_fake,
        "system_prompt": systemPrompt,
        "user_prompt": prompt,
        "one_shot": oneShot,
        #"imageBoost": imageBoost
    }
    if oneShot:
        results["one_shot_image"] = exampleImage
        results["tag_shot_image"] = labExample
    return results


def saveAllJson(metrics, responses, PromptITA, modelName, i):
    outputData = {
        "metrics": metrics,
        "responses": responses
    }
    Path("resultsJSON").mkdir(exist_ok=True)

    # Imposta lingua
    language_tag = "ITA" if PromptITA else "ENG"

    # Timestamp per identificare diversi tentativi
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")

    # Pulisci MODEL_NAME da caratteri non ammessi nei nomi file
    safe_model_name = modelName.replace(":", "_").replace("/", "_")
    
    # Costruisci filename
    filename = f"resultsJSON/real-vs-fake_{safe_model_name}_PromptType-{i}_{language_tag}_{timestamp}_result.json"
    # Salva JSON
    with open(filename, "w") as f:
        json.dump(outputData, f, indent=4)

    print(f"Results saved to {filename}.")

### Valori modificabili

In [9]:
# VALORI PER CARICARE IL DATASET
saveDatasetDirectory = False
startMiniDt = False
NAME = "oneshot/test_OS"
MAX_IMAGES = 300
SHUFFLE = False
# ======================================

# VALORI PER IL PROMPT
INDEX_PROMPT = 3  # (0-6)
IS_ITALIAN = False
SHOW_IMAGES = False
UNCERTAIN_EN = True
IMAGE_BOOST= False
# ===================================
# VALORI PER IL MODELLO

MODEL_NAME = "qwen2.5VL:3B"
# ===================================
# MODALITA' AUTOMATICA

AUTO_ON = False
# ===================================
# MODALITA' ONESHOT

ONESHOT = False
TAG_IMAGE = True
ONE_SHOT_IMAGE="/kaggle/input/oneshot/photoEx/real/woman3-4.jpg"
# =======================================
# MODALITA' FEWSHOT

FEWSHOT = False
FakeExample = "/kaggle/input/oneshot/photoEx/fake/difficultwoman.jpg"
RealExample = "/kaggle/input/oneshot/photoEx/real/woman3-4.jpg"

### MAIN

In [12]:

if startMiniDt:
    images_with_labels, fakes, reals = loadDataset(MAX_IMAGES)
    saveDataset(images_with_labels, NAME)
else:
    images_with_labels, fakes, reals = loadExistingDataset(NAME)
    

if SHUFFLE:
    shuffleDataset(images_with_labels)
if TAG_IMAGE:
    isFake = False
else:
    isFake = True
# prompt section
#oneShotMessage = None
#if ONESHOT:
#    oneShotMessage = usingOneShot()
if AUTO_ON:
    DATASETS = ["oth-test/test_2", "oth-test/test_3"]
    for dataset in DATASETS:
        print(f"\n=== Running dataset: {dataset} ===")

        images_with_labels, fakes, reals = loadExistingDataset(dataset)
        if SHUFFLE:
            shuffleDataset(images_with_labels)
        for INDEX_PROMPT in range(7):  # Prompt da 0 a 6
            for IS_ITALIAN in [False, True]:  # Prima inglese, poi italiano
                userPrompt = chooseAPrompt(INDEX_PROMPT, IS_ITALIAN)
                systemPrompt = getSystemPrompt(IS_ITALIAN, UNCERTAIN_EN)
    
                print(f"\n--- Prompt {INDEX_PROMPT} | Language: {'Italian' if IS_ITALIAN else 'English'} ---")
                print("Prompt:", userPrompt)
    
                # Inizializza le metriche
                counters = initMetrics()
                results = []
    
                # Ciclo principale di analisi immagini
                for img_path, label in tqdm(images_with_labels, desc=f"Analyzing images (Prompt {INDEX_PROMPT}, {'IT' if IS_ITALIAN else 'EN'})"):
                    print(img_path)
                    result, counters = analyze_image(
                        img_path, label, userPrompt, MODEL_NAME,
                        ONESHOT, systemPrompt,counters, SHOW_IMAGES, 
                        ONE_SHOT_IMAGE, TAG_IMAGE, IS_ITALIAN
                    )
                    results.append(result)
    
                # Salvataggio dei risultati
                jsonINFO = analizeMetrics(
                    counters, images_with_labels, userPrompt,
                    systemPrompt, ONESHOT, ONE_SHOT_IMAGE, TAG_IMAGE,
                    reals, fakes, IMAGE_BOOST, 
                )
    
                saveAllJson(jsonINFO, results, IS_ITALIAN, MODEL_NAME, INDEX_PROMPT)

else:
    userPrompt = chooseAPrompt(INDEX_PROMPT, IS_ITALIAN)
    systemPrompt = getSystemPrompt(IS_ITALIAN, UNCERTAIN_EN)
    
    print("you choose " + userPrompt)
    # initialize metrics
    counters = initMetrics()
    results = []
    # Main analysis loop
    for img_path, label in tqdm(images_with_labels, desc=" Analyzing images"):
        print(img_path)
        result, counters = analyze_image(
                    img_path, label, userPrompt, MODEL_NAME,
                    ONESHOT, systemPrompt,counters, SHOW_IMAGES, 
                    ONE_SHOT_IMAGE, isFake, IS_ITALIAN
                )
        results.append(result)
    jsonINFO = analizeMetrics(
                counters, images_with_labels, userPrompt,
                systemPrompt, ONESHOT, ONE_SHOT_IMAGE, TAG_IMAGE,
                reals, fakes, IMAGE_BOOST, 
            )
    saveAllJson(jsonINFO, results, IS_ITALIAN, MODEL_NAME, INDEX_PROMPT)

you choose Is the provided image showing a real face or a generated one? I will pay you $1,000 for the correct answer.


 Analyzing images:   0%|          | 0/300 [00:00<?, ?it/s]

/kaggle/input/oneshot/test_OS/real/real_111.jpg


 Analyzing images:   0%|          | 1/300 [00:09<49:01,  9.84s/it]


Image: real_111.jpg
Raw Output: ['{\n  "result": "[uncertain]",\n  "explanation": "The image shows a close-up of a person\'s face with visible skin texture and lighting effects. However, the quality and clarity of the image suggest it could be a generated photo due to the smoothness and lack of natural imperfections typically found in real faces. Additionally, the lighting and shadows do not appear to be consistent with natural light sources."\n}']
 Rejection on real image
/kaggle/input/oneshot/test_OS/real/real_59.jpg


...

 Analyzing images: 100%|█████████▉| 299/300 [34:42<00:06,  6.90s/it]


Image: fake_165.jpg
Raw Output: ['{\n  "result": "[generated]",\n  "explanation": "The image shows a person wearing a red scarf and a dark jacket with a patterned background. The lighting and shadows suggest a controlled environment, typical of studio photography. The skin texture, lighting, and overall appearance do not align with natural human features, indicating that this is likely a generated image."\n}']
 TP (fake correctly identified)
/kaggle/input/oneshot/test_OS/fake/fake_227.jpg


 Analyzing images: 100%|██████████| 300/300 [34:49<00:00,  6.96s/it]


Image: fake_227.jpg
Raw Output: ['{\n  "result": "[uncertain]",\n  "explanation": "The image appears to be of a child with a bright smile and clear skin. However, the lighting and shadows suggest it might be a generated image due to the uniformity and lack of natural variations typically found in real faces. Additionally, the clarity and sharpness of the features could indicate a digital creation rather than a photograph."\n}']
 Rejection on fake image

====== FINAL REPORT ======
Total processed: 300
TP: 27 | TN: 8 | FP: 46 | FN: 12
Rejections on real: 95 | Rejections on fake: 111
Text parsing errors: 1 (0.33%)

Accuracy: 0.3763
Precision: 0.3699
Recall: 0.6923
False Negative Rate (real->fake): 8.96%
False Positive Rate (fake->real): 27.88%
Rejection Rate on real images: 70.90%
Rejection Rate on fake images: 67.27%
One-class Accuracy (real images): 0.1481
One-class Accuracy (fake images): 0.6923


Results saved to resultsJSON/real-vs-fake_qwen2.5VL_3B_PromptType-3_ENG_20250906-104757_result.json.


## Informazioni generali

Questo notebook serve per analizzare e fare prompt engineering sul modello qwen2.5VL:3b per testare quale prompt è più favorevole all'identificare immagini vere o false. Usiamo kaggle per la facilità di inserire dataset e soprattutto per la sua velocità ad analizzare le immagini. Tuttavia, non è compatibile con OLLAMA e quindi si usa Hugging Face